<a href="https://colab.research.google.com/github/ZeyadSabbah/TrivagoRecommenderSystem/blob/master/KMeansPCAModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Selection & Modeling
## Mounting to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem

/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem


## Loading Libraries & Datasets

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import random
import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
TrainDataFilepath = './Datasets/clean_data/Sets/train.csv'
TrainData = pd.read_csv(TrainDataFilepath)
TrainData.dropna(inplace=True)

#declaring features and label
features = TrainData.drop(columns=['session_id', 'item_id', 'clickout']).columns.tolist()
label = ['clickout']

FeaturesToDrop = ['NumberInImpressions', 'NumberInReferences', 'MeanPrice', 'MinPrice']
for feature in FeaturesToDrop:
  features.remove(feature)

X_train = TrainData[features]
y_train = TrainData[label]

TrainData_sessions_item = TrainData[['session_id', 'item_id', 'clickout']]

TrainData is ready for processing and modeling, while validation and test sets still need to be engineered. There is a ready function that will transform sets into the same form of TrainData.
## Scaling Features

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', StandardScaler()),
])

from sklearn.compose import ColumnTransformer
full_pipeline = ColumnTransformer([
("num", num_pipeline, list(X_train))
])

X_train_scaled = full_pipeline.fit_transform(X_train)

## Properties

In [0]:
def ConcatProperties(data, ClustersOrPCA, X_scaled):
  '''
  Desc: function that concatenates either clusters or PCA of properties

  Input: data: Pandas DataFrame with the data being used to concatenate properties to
         ClustersOrPCA: String with which type to be used of properties
         X_scaled: numpy array of the features of the dataframe used
         
  Output: X_scaled_prop: Pandas DataFrame with the properties of different items being concatenated to the features dataframe
  '''
  if ClustersOrPCA == 'Clusters':
    PropertiesFilePath = './Datasets/clean_data/ItemsPropertiesSimilarities/PropertiesClusters.csv'
    DF = pd.read_csv(PropertiesFilePath)
    DF = DF.rename(columns={'item':'item_id'})
  elif ClustersOrPCA == 'PCA':
    PropertiesFilePath = './Datasets/clean_data/ItemsPropertiesSimilarities/PCA_Properties.csv'
    DF = pd.read_csv(PropertiesFilePath)
    DF.drop(columns='Unnamed: 0', inplace=True)
  X_scaled = pd.DataFrame(X_scaled, columns=features)
  X_scaled['item_id'] = data['item_id'].values.tolist()
  X_scaled_prop = X_scaled.merge(DF, on='item_id', how='left')
  if ClustersOrPCA == 'Clusters':
    X_scaled_prop.fillna(0, inplace=True)
  elif ClustersOrPCA == 'PCA':
    X_scaled_prop.fillna(X_scaled_prop.mean())
  X_scaled_prop.drop(columns='item_id', inplace=True)
  return X_scaled_prop

 #### Clusters

In [14]:
X_train_Prop_clust = ConcatProperties(TrainData,'Clusters', X_train_scaled)
X_train_Prop_clust.head(2)

,price,item_rank,price_rank,session_duration,item_duration,item_session_duration,item_interactions,maximum_step,top_list,NumberOfProperties,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MaxPrice,AveragePriceRank,item_id,A,B,C,D,E
0,0.252317,-1.600463,-0.482369,0.292219,-0.05385,-0.109292,-0.121237,-0.033225,1.892137,0.906773,0.285278,0.182823,-0.287512,-0.452075,0.050683,-0.046869,1.709832,0.175147,55109,1,0,0,0,0
1,-0.547093,-1.460681,-1.460852,0.292219,-0.05385,-0.109292,-0.121237,-0.033225,1.892137,-0.566203,-0.197544,-0.216819,0.043623,-0.288748,0.064645,-0.370136,-0.134111,-0.583259,129343,1,0,0,0,0


### PCA

In [15]:
X_train_Prop_PCA = ConcatProperties(TrainData, 'PCA', X_train_scaled)
X_train_Prop_PCA.head(2)

,price,item_rank,price_rank,session_duration,item_duration,item_session_duration,item_interactions,maximum_step,top_list,NumberOfProperties,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MaxPrice,AveragePriceRank,item_id,Unnamed: 0,PC1,PC2,PC3,PC4,PC5,PC6,PC7
0,0.252317,-1.600463,-0.482369,0.292219,-0.05385,-0.109292,-0.121237,-0.033225,1.892137,0.906773,0.285278,0.182823,-0.287512,-0.452075,0.050683,-0.046869,1.709832,0.175147,55109,917942,4.256183,-1.010829,0.194052,-0.343445,-0.483854,0.329065,1.082989
1,-0.547093,-1.460681,-1.460852,0.292219,-0.05385,-0.109292,-0.121237,-0.033225,1.892137,-0.566203,-0.197544,-0.216819,0.043623,-0.288748,0.064645,-0.370136,-0.134111,-0.583259,129343,657696,0.936076,0.037035,0.952187,0.495297,-0.157769,-0.341891,-0.289848


## XGBoost Modeling

Since XGBoost without Resampling reached the highest score, then this algorigthm will be the one functioning on both Properties clusters and PCA.

In [0]:
def random_search(clf, parameters, X, y):
  '''
  Desc: function that searches for the best hyperparameters for an algorithm by fitting randomly training example randomly and select the best

  Input: clf: classifier algorithm
         parameters: list of different parameters
  
  Output: best_clf: classifier with the best hyperparameters that suit this kind of data examples
  '''
  search_obj = RandomizedSearchCV(clf, parameters)
  fit_obj = search_obj.fit(X, y)
  best_clf = fit_obj.best_estimator_
  return best_clf

#### Clusters

In [15]:
X_train_Prop_clust_sample = X_train_Prop_clust.sample(frac=0.02, random_state=0)
y_train_sample = y_train.sample(frac=0.02, random_state=0)

t1 = datetime.now()
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators':[5, 50, 250, 500],
    'max_depth':[1, 3, 5, 7, 9],
    'learning_rate':[0.01, 0.1, 1, 10, 100]
}

best_clf_grid = random_search(gb, parameters, X_train_Prop_clust_sample, y_train_sample.values.ravel())
print(best_clf_grid)

gb = best_clf_grid.fit(X_train_Prop_clust, y_train.values.ravel())
joblib.dump(gb, './modelsProperties/XGBoost_clusters.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=250,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
Time taken :  4:25:56.523560


#### PCA

In [0]:
X_train_Prop_PCA_sample = X_train_Prop_PCA.sample(frac=0.02, random_state=0)
y_train_sample = y_train.sample(frac=0.02, random_state=0)

t1 = datetime.now()
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators':[5, 50, 250, 500],
    'max_depth':[1, 3, 5, 7, 9],
    'learning_rate':[0.01, 0.1, 1, 10, 100]
}

best_clf_grid = random_search(gb, parameters, X_train_Prop_PCA_sample, y_train_sample.values.ravel())
print(best_clf_grid)

gb = best_clf_grid.fit(X_train_Prop_PCA, y_train.values.ravel())
joblib.dump(gb, './modelsProperties/XGBoost_PCA.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

## Model Evaluation

In [0]:
#function is from this repo https://gist.github.com/bwhite/3726239
def mean_reciprocal_rank(rs):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105
    >>> rs = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
    >>> mean_reciprocal_rank(rs)
    0.5
    >>> rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.75
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

In [0]:
def get_probabilities(model_path, X, session_item_dataset):
  '''
  Desc: function that gets the probability of each item being selected by the user, rerank the items in the session based on the probabilites

  Input: model_path: String with the name of the stored model
         X: array of scaled features of the dataset
         session_item_dataset: Pandas Dataframe with the sessions, items, and clickout
        
  Output: clickout_rank: List of lists that carries which item was selected in which rank
          RecommendationsDF: Pandas Dataframe to be transformed and merged to the Clickout Dataframe
  '''
  model = joblib.load(model_path)
  BothProbabilities = model.predict_proba(X)
  Probabilities = [Probability[1] for Probability in BothProbabilities]
  session_item_dataset['probability'] = Probabilities
  RecommendationsDF = session_item_dataset.groupby(['session_id'], sort=False).apply(lambda x: (x.sort_values('probability', ascending=False)))
  clickout_rank = RecommendationsDF.clickout
  clickout_rank = clickout_rank.reset_index().groupby('session_id').clickout.apply(list).values.tolist()
  return clickout_rank, RecommendationsDF

In [0]:
def ClassifReport(model_path, X, y):
  global y_pred
  model = joblib.load(model_path)
  y_pred = model.predict(X)
  return classification_report(y, y_pred)

In [0]:
def PrintMetrics(model_path, X, y, session_item_dataset):
  clickout_rank, RecommendationsDF = get_probabilities(model_path, X, session_item_dataset)
  MeanReciprocalRank = mean_reciprocal_rank(clickout_rank)
  print('Mean Reciprocal Rank : ', MeanReciprocalRank)
  print('=================================================')
  ClassificationReport = ClassifReport(model_path, X, y)
  print('Classification Report')
  print('=================================================')
  print(ClassificationReport)
  ConfMatrix = confusion_matrix(y, y_pred)
  print('Confusion Matrix')
  print('================================================')
  print(ConfMatrix)
  return

### Loading and Preparing Validation & Test Sets

#### Validation

In [0]:
valFilepath = './Datasets/clean_data/Sets/val.csv'
valData = pd.read_csv(valFilepath)

#declaring features and label
features = valData.drop(columns=['session_id', 'item_id', 'clickout']).columns.tolist()
label = ['clickout']

FeaturesToDrop = ['NumberInImpressions', 'NumberInReferences', 'MeanPrice', 'MinPrice']
for feature in FeaturesToDrop:
  features.remove(feature)

valData_sessions_item = valData[['session_id', 'item_id', 'clickout']]
X_val = valData[features]
y_val = valData[label]

# validation set scaling
X_val_scaled = full_pipeline.fit_transform(X_val)

X_val_Prop_clust = ConcatProperties(valData, 'Clusters', X_val_scaled)
X_val_Prop_PCA = ConcatProperties(valData, 'PCA', X_val_scaled)

In [32]:
X_val_Prop_clust

,price,item_rank,price_rank,session_duration,item_duration,item_session_duration,item_interactions,maximum_step,top_list,NumberOfProperties,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MaxPrice,AveragePriceRank,A,B,C,D,E
0,-0.196916,-1.600155,-1.180739,1.891658,20.486682,13.531549,1.696958,-0.037139,1.891705,-1.581095,0.649658,0.413817,-0.047192,0.117955,-0.038701,0.262537,-0.215300,0.603357,0.0,0.0,0.0,0.0,1.0
1,-0.185278,-1.460350,0.077509,1.891658,-0.060067,-0.109652,-0.125926,-0.037139,1.891705,0.307928,0.287566,0.356729,-0.130984,-0.192631,0.446863,-0.350283,-0.040253,0.156429,1.0,0.0,0.0,0.0,0.0
2,0.018393,-1.320545,-1.040934,1.891658,-0.060067,-0.109652,-0.125926,-0.037139,1.891705,-0.797841,-0.134874,-0.271240,-0.460867,-0.418206,-0.455428,0.689664,0.318351,0.672071,1.0,0.0,0.0,0.0,0.0
3,-0.260927,-1.180739,-0.621518,1.891658,-0.060067,-0.109652,-0.125926,-0.037139,1.891705,-0.106735,0.076346,-0.157064,-0.315359,-0.418977,-0.439565,0.137521,-0.293099,0.222219,1.0,0.0,0.0,0.0,0.0
4,-0.231831,-1.040934,-1.600155,1.891658,-0.060067,-0.109652,-0.125926,-0.037139,1.891705,-1.535021,-0.195223,-0.157064,0.462749,0.576486,0.432903,0.250411,-0.381838,0.349947,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399261,0.478108,1.195951,1.335756,-0.427598,-0.060067,-0.109652,-0.125926,-0.337035,-0.528624,0.538297,0.679832,0.642169,0.015135,-0.308736,0.258409,0.277854,0.329291,0.307601,1.0,0.0,0.0,0.0,0.0
3399262,1.205505,1.335756,-0.621518,-0.427598,-0.060067,-0.109652,-0.125926,-0.337035,-0.528624,-0.152809,9.913173,9.205379,0.561929,-0.301944,0.231018,-1.447271,10.073586,-0.132933,1.0,0.0,0.0,0.0,0.0
3399263,0.402459,1.475562,0.776535,-0.427598,-0.060067,-0.109652,-0.125926,-0.337035,-0.528624,-1.581095,0.347915,0.128376,-0.204116,0.842446,-0.148743,0.122994,0.875098,0.239691,0.0,0.0,0.0,0.0,1.0
3399264,3.125834,1.615367,-0.481713,-0.427598,-0.060067,-0.109652,-0.125926,-0.337035,-0.528624,1.183329,-0.406443,-0.385416,-0.279705,-0.432118,0.258409,0.891143,0.941956,0.629831,1.0,0.0,0.0,0.0,0.0


#### Test

In [0]:
testFilepath = './Datasets/clean_data/Sets/test.csv'
testData = pd.read_csv(testFilepath)

FeaturesToDrop = ['NumberInImpressions', 'NumberInReferences', 'MeanPrice', 'MinPrice']
for feature in FeaturesToDrop:
  features.remove(feature)

testData_sessions_item = testData[['session_id', 'item_id', 'clickout']]
X_test = testData[features]
y_test  = testData[label]

# test set scaling
X_test_scaled = full_pipeline.fit_transform(X_test)

X_test_Prop_clust = ConcatProperties(testData, 'Clusters', X_test_scaled)
X_test_Prop_PCA = ConcatProperties(testData, 'PCA', X_test_scaled)

### Clusters Evaluation

In [31]:
PrintMetrics('./modelsProperties/XGBoost_clusters.pkl', X_val_Prop_clust, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5960285069151403
Classification Report
              precision    recall  f1-score   support

           0       0.96      1.00      0.98   3249590
           1       0.63      0.14      0.23    149676

    accuracy                           0.96   3399266
   macro avg       0.80      0.57      0.61   3399266
weighted avg       0.95      0.96      0.95   3399266

Confusion Matrix
[[3237239   12351]
 [ 128260   21416]]


###PCA Evaluation

In [0]:
PrintMetrics('./modelsProperties/XGBoost_clusters.pkl', X_val_Prop_PCA, y_val, valData_sessions_item)